# Installing libraries and modules

In [1]:
!pip install optuna
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install pysentimiento

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 12.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 

In [2]:
from google.colab import drive
from  IPython. display import clear_output
import optuna
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np
import random
import  matplotlib. pyplot  as  plt
from tqdm import tqdm
import torch
import torch.optim as optim
import torch. nn.functional as F
import re
import warnings
warnings. filterwarnings('ignore')

In [6]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train_all_df = pd.read_csv('/content/drive/MyDrive/EXIST 2024/Final/Tweet/Train/all/final_tweet_train_all.csv')
train_en_df = pd.read_csv('/content/drive/MyDrive/EXIST 2024/Final/Tweet/Train/en/final_tweet_train_en.csv')
train_es_df = pd.read_csv('/content/drive/MyDrive/EXIST 2024/Final/Tweet/Train/es/final_tweet_train_es.csv')

dev_all_df = pd.read_csv('/content/drive/MyDrive/EXIST 2024/Final/Tweet/Dev/all/final_tweet_dev_all.csv')
dev_en_df = pd.read_csv('/content/drive/MyDrive/EXIST 2024/Final/Tweet/Dev/en/final_tweet_dev_en.csv')
dev_es_df = pd.read_csv('/content/drive/MyDrive/EXIST 2024/Final/Tweet/Dev/es/final_tweet_dev_es.csv')

In [8]:
device = torch. device('cuda:0'  if  torch.cuda.is_available() else  'cpu')
print(f"computation will run on {device} now")

computation will run on cuda:0 now


In [20]:
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@user", t)
    t = URL_RE.sub("http", t)
    tweets.append(t)
  return tweets


# Transformers



## All:
- XLM_RoBERTa
- cardiffnlp/twitter-xlm-roberta-base
- Twitter-RoBERTa-base-hate

## Spanish:
- RoberTuito
- BETO
- hackathon-somos-nlp-2023/roberta-base-bne-finetuned-suicide-es
- edu- munozsala/roberta_bne_sentiment_analysis_es

## English:
- NLP-LTU/bertweet-large-sexism-detector
- NLP-LTU/distilbert-sexism-detector

# Construct Optuna - For all tasks hard

In [ ]:
train_all_df.head()

,Unnamed: 0,id,lang,tweet,value_ttgt1s_data,value_ttgt1h_data,value_ttgt2s_data,value_ttgt2h_data,value_ttgt3s_data,value_ttgt3h_data
0,0,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...","{""YES"": 0.8333333333333334, ""NO"": 0.1666666666...","""YES""","{""REPORTED"": 0.5, ""JUDGEMENTAL"": 0.33333333333...","""REPORTED""","{""OBJECTIFICATION"": 0.3333333333333333, ""SEXUA...","[""OBJECTIFICATION"", ""SEXUAL-VIOLENCE""]"
1,1,100002,es,@ultimonomada_ Si comicsgate se parece en algo...,"{""NO"": 0.8333333333333334, ""YES"": 0.1666666666...","""NO""","{""NO"": 0.8333333333333334, ""DIRECT"": 0.1666666...","""NO""","{""NO"": 0.8333333333333334, ""OBJECTIFICATION"": ...","[""NO""]"
2,2,100003,es,"@Steven2897 Lee sobre Gamergate, y como eso ha...","{""NO"": 1.0, ""YES"": 0.0}","""NO""","{""NO"": 1.0, ""DIRECT"": 0.0, ""REPORTED"": 0.0, ""J...","""NO""","{""NO"": 1.0, ""IDEOLOGICAL-INEQUALITY"": 0.0, ""ST...","[""NO""]"
3,3,100004,es,@Lunariita7 Un retraso social bastante lamenta...,"{""NO"": 0.5, ""YES"": 0.5}",NaN,"{""NO"": 0.5, ""DIRECT"": 0.16666666666666666, ""RE...",NaN,"{""NO"": 0.5, ""IDEOLOGICAL-INEQUALITY"": 0.5, ""OB...",NaN
4,4,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,"{""YES"": 0.6666666666666666, ""NO"": 0.3333333333...","""YES""","{""REPORTED"": 0.16666666666666666, ""NO"": 0.3333...",NaN,"{""STEREOTYPING-DOMINANCE"": 0.5, ""OBJECTIFICATI...","[""STEREOTYPING-DOMINANCE"", ""OBJECTIFICATION"", ..."


## task 1 hard

### All

In [ ]:
t1_train_all_df_sc = train_all_df.loc[:, ['id', 'lang', 'tweet', 'value_ttgt1s_data', 'value_ttgt1h_data']]
t1_train_all_df_sc.head()

,id,lang,tweet,value_ttgt1s_data,value_ttgt1h_data
0,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...","{""YES"": 0.8333333333333334, ""NO"": 0.1666666666...","""YES"""
1,100002,es,@ultimonomada_ Si comicsgate se parece en algo...,"{""NO"": 0.8333333333333334, ""YES"": 0.1666666666...","""NO"""
2,100003,es,"@Steven2897 Lee sobre Gamergate, y como eso ha...","{""NO"": 1.0, ""YES"": 0.0}","""NO"""
3,100004,es,@Lunariita7 Un retraso social bastante lamenta...,"{""NO"": 0.5, ""YES"": 0.5}",NaN
4,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,"{""YES"": 0.6666666666666666, ""NO"": 0.3333333333...","""YES"""


In [ ]:
t1_train_all_df_sc.rename(columns={'value_ttgt1s_data': 'soft_label', 'value_ttgt1h_data': 'hard_label'}, inplace=True)

In [ ]:
t1_train_all_df_sc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6920 entries, 0 to 6919
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          6920 non-null   int64 
 1   lang        6920 non-null   object
 2   tweet       6920 non-null   object
 3   soft_label  6920 non-null   object
 4   hard_label  6064 non-null   object
dtypes: int64(1), object(4)
memory usage: 270.4+ KB


In [13]:
t1_dev_all_df_sc = dev_all_df.loc[:, ['id', 'lang', 'tweet', 'value_tdgt1s_data', 'value_tdgt1h_data']]
t1_dev_all_df_sc.head()

,id,lang,tweet,value_tdgt1s_data,value_tdgt1h_data
0,300001,es,@Fichinescu La comunidad gamer es un antro de ...,"{""NO"": 0.5, ""YES"": 0.5}",NaN
1,300002,es,@anacaotica88 @MordorLivin No me acuerdo de lo...,"{""YES"": 0.8333333333333334, ""NO"": 0.1666666666...","""YES"""
2,300003,es,@cosmicJunkBot lo digo cada pocos dias y lo re...,"{""NO"": 1.0, ""YES"": 0.0}","""NO"""
3,300004,es,Also mientras les decia eso la señalaba y deci...,"{""NO"": 0.16666666666666666, ""YES"": 0.833333333...","""YES"""
4,300005,es,"And all people killed, attacked, harassed by ...","{""NO"": 0.8333333333333334, ""YES"": 0.1666666666...","""NO"""


In [14]:
t1_dev_all_df_sc.rename(columns={'value_tdgt1s_data': 'soft_label', 'value_tdgt1h_data': 'hard_label'}, inplace=True)
t1_dev_all_df_sc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038 entries, 0 to 1037
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1038 non-null   int64 
 1   lang        1038 non-null   object
 2   tweet       1038 non-null   object
 3   soft_label  1038 non-null   object
 4   hard_label  934 non-null    object
dtypes: int64(1), object(4)
memory usage: 40.7+ KB


`cardiffnlp/twitter-xlm-roberta-base`

In [15]:
def task1_hard_encode(df):
    task1_encoder.fit(all_task1_hard_labels)
    df['hard_label'] = task1_encoder.transform(df['hard_label'])
    return df

def task1_hard_decode(df):
    df["hard_label"] = task1_encoder.inverse_transform(df["hard_label"])
    return df

In [ ]:
#instantiate label encoders
from sklearn.preprocessing import LabelEncoder
task1_encoder = LabelEncoder()
task2_encoder = LabelEncoder()
task3_encoder = LabelEncoder()


og_train1 = t1_train_all_df_sc.copy()
og_dev1 = t1_dev_all_df_sc.copy()

all_task1_hard_labels = pd.concat([og_train1["hard_label"],og_dev1["hard_label"]])

train1_df = task1_hard_encode(og_train1)
# print(train1_df.columns)
train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df = train1_df[train1_df['hard_label'] != 2]
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

test1_df = task1_hard_encode(og_dev1)
test1_df = test1_df[["tweet","hard_label"]].dropna()
test1_df = test1_df[test1_df['hard_label'] != 2]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)


train1 (6064, 2)
<bound method NDFrame.head of                                                   tweet  hard_label
0     @USER Ignora al otro, es un capullo.El problem...           1
1     @USER Si comicsgate se parece en algo a gamerg...           0
2     @USER Lee sobre Gamergate, y como eso ha cambi...           0
4     @USER @USER @USER Entonces como así es el merc...           1
5     @USER Aaah sí. Andrew Dobson. El que se dedicó...           0
...                                                 ...         ...
6915  idk why y’all bitches think having half your a...           1
6916  This has been a part of an experiment with @US...           1
6917  "Take me already" "Not yet. You gotta be ready...           1
6918    @USER why do you look like a whore? /lh HTTPURL           1
6919  ik when mandy says “you look like a whore” i l...           1

[6064 rows x 2 columns]>
test1 (934, 2)
<bound method NDFrame.head of                                                   tweet  hard_labe

In [ ]:
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
# Load the tokenizer and model
model_name = "cardiffnlp/twitter-xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/EXIST 2024/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_all_hard'
MAX_LENGTH = 128

In [ ]:
from datasets import Dataset
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

In [ ]:
# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)

In [ ]:
# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

In [ ]:
def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-all-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

[I 2024-04-13 08:20:26,683] A new study created in memory with name: hp-search-task1-all-hard


Step,Training Loss
500,0.719400
1000,0.700400
1500,0.693200


Step,Training Loss
500,0.719400
1000,0.700400
1500,0.693200
2000,0.694200
2500,0.692700


[I 2024-04-13 08:34:59,081] Trial 0 finished with value: 0.6984509440830776 and parameters: {'learning_rate': 0.0005052522866587834, 'weight_decay': 0.002491147101567152, 'num_train_epochs': 4}. Best is trial 0 with value: 0.6984509440830776.


In [ ]:
print('--------------------- Finding study best parameters ------------------ \n')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print('------------------ Extract best study params ---------------------- \n')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print('------------------- Create dictionary of the best hyperparameters ------------------ \n')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

--------------------- Finding study best parameters ------------------ 

------------------ Extract best study params ---------------------- 

The best learning rate is: 0.0005052522866587834
The best weight decay is: 0.002491147101567152
The best epoch is : 4
------------------- Create dictionary of the best hyperparameters ------------------ 



In [ ]:
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 4
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/EXIST 2024/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_all_hard_with_sdadas'
MAX_LENGTH = 128

In [ ]:
# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)

In [ ]:
# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

In [ ]:
import os
import tempfile

def objective(trial):
    # temp_dir = tempfile.mkdtemp()
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    # os.rmdir(temp_dir)
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-all-hard-with-sdadas', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

[I 2024-04-13 09:34:56,907] A new study created in memory with name: hp-search-task1-all-hard-with-sdadas


Step,Training Loss
500,0.059500
1000,0.041100
1500,0.023100
2000,0.019900


Step,Training Loss
500,0.059500
1000,0.041100
1500,0.023100
2000,0.019900
2500,0.005400


[I 2024-04-13 10:14:50,853] Trial 0 finished with value: 0.028291460531098503 and parameters: {'learning_rate': 5.4734446759247935e-06, 'weight_decay': 0.004881728253476856, 'num_train_epochs': 4}. Best is trial 0 with value: 0.028291460531098503.


In [ ]:
print('--------------------- Finding study best parameters ------------------ \n')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print('------------------ Extract best study params ---------------------- \n')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print('------------------- Create dictionary of the best hyperparameters ------------------ \n')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

--------------------- Finding study best parameters ------------------ 

------------------ Extract best study params ---------------------- 

The best learning rate is: 5.4734446759247935e-06
The best weight decay is: 0.004881728253476856
The best epoch is : 4
------------------- Create dictionary of the best hyperparameters ------------------ 



### EN

`NLP-LTU/bertweet-large-sexism-detector`

In [9]:
t1_train_en_df_sc = train_en_df.loc[:, ['id', 'lang', 'tweet', 'value_ttgt1s_data', 'value_ttgt1h_data']]
t1_train_en_df_sc.head()

,id,lang,tweet,value_ttgt1s_data,value_ttgt1h_data
0,100001,es,"@TheChiflis Ignore the other one, he's an idio...","{""YES"": 0.8333333333333334, ""NO"": 0.1666666666...","""YES"""
1,100002,es,@ultimonomada_ If comicsgate is anything like ...,"{""NO"": 0.8333333333333334, ""YES"": 0.1666666666...","""NO"""
2,100003,es,"@Steven2897 Read about Gamergate, and how it h...","{""NO"": 1.0, ""YES"": 0.0}","""NO"""
3,100004,es,"@Lunariita7 A quite unfortunate social delay, ...","{""NO"": 0.5, ""YES"": 0.5}",NaN
4,100005,es,@novadragon21 @icep4ck @TvDannyZ So since this...,"{""YES"": 0.6666666666666666, ""NO"": 0.3333333333...","""YES"""


In [11]:
t1_train_en_df_sc.rename(columns={'value_ttgt1s_data': 'soft_label', 'value_ttgt1h_data': 'hard_label'}, inplace=True)

In [18]:
def task1_hard_encode(df):
    task1_encoder.fit(en_task1_hard_labels)
    df['hard_label'] = task1_encoder.transform(df['hard_label'])
    return df

def task1_hard_decode(df):
    df["hard_label"] = task1_encoder.inverse_transform(df["hard_label"])
    return df

In [21]:
#instantiate label encoders
from sklearn.preprocessing import LabelEncoder
task1_encoder = LabelEncoder()
task2_encoder = LabelEncoder()
task3_encoder = LabelEncoder()


og_train1 = t1_train_en_df_sc.copy()
og_dev1 = t1_dev_all_df_sc.copy()

en_task1_hard_labels = pd.concat([og_train1["hard_label"],og_dev1["hard_label"]])

train1_df = task1_hard_encode(og_train1)
# print(train1_df.columns)
train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df = train1_df[train1_df['hard_label'] != 2]
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

test1_df = task1_hard_encode(og_dev1)
test1_df = test1_df[["tweet","hard_label"]].dropna()
test1_df = test1_df[test1_df['hard_label'] != 2]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)

train1 (6064, 2)
<bound method NDFrame.head of                                                   tweet  hard_label
0     @user Ignore the other one, he's an idiot. The...           1
1     @user If comicsgate is anything like gamergate...           0
2     @user Read about Gamergate, and how it has cha...           0
4     @user @user @user So since this is the market,...           1
5     @user Aaah yes. Andrew Dobson. The one who ded...           0
...                                                 ...         ...
6915  idk why y’all bitches think having half your a...           1
6916  This has been a part of an experiment with @us...           1
6917  "Take me already" "Not yet. You gotta be ready...           1
6918       @user why do you look like a whore? /lh http           1
6919  ik when mandy says “you look like a whore” i l...           1

[6064 rows x 2 columns]>
test1 (934, 2)
<bound method NDFrame.head of                                                   tweet  hard_labe

In [22]:
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [23]:
# Load the tokenizer and model
model_name = "NLP-LTU/bertweet-large-sexism-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [24]:
# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/EXIST 2024/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_en_hard'
MAX_LENGTH = 128

In [25]:
from datasets import Dataset
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

In [26]:
# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)

In [27]:
# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

In [28]:
def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-en-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

[I 2024-04-15 07:43:37,846] A new study created in memory with name: hp-search-task1-en-hard


Step,Training Loss
500,0.456600
1000,0.337400
1500,0.303900
2000,0.224700
2500,0.191000
3000,0.139300
3500,0.117000


[I 2024-04-15 08:27:32,863] Trial 0 finished with value: 0.25284674290248327 and parameters: {'learning_rate': 4.335929457904673e-06, 'weight_decay': 0.00011999774854545356, 'num_train_epochs': 5}. Best is trial 0 with value: 0.25284674290248327.


In [29]:
print('--------------------- Finding study best parameters ------------------ \n')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print('------------------ Extract best study params ---------------------- \n')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print('------------------- Create dictionary of the best hyperparameters ------------------ \n')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

--------------------- Finding study best parameters ------------------ 

------------------ Extract best study params ---------------------- 

The best learning rate is: 4.335929457904673e-06
The best weight decay is: 0.00011999774854545356
The best epoch is : 5
------------------- Create dictionary of the best hyperparameters ------------------ 



### ES

`pysentimiento/robertuito-base-uncased`

In [10]:
t1_train_es_df_sc = train_es_df.loc[:, ['id', 'lang', 'tweet', 'value_ttgt1s_data', 'value_ttgt1h_data']]
t1_train_es_df_sc.head()

,id,lang,tweet,value_ttgt1s_data,value_ttgt1h_data
0,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...","{""YES"": 0.8333333333333334, ""NO"": 0.1666666666...","""YES"""
1,100002,es,@ultimonomada_ Si comicsgate se parece en algo...,"{""NO"": 0.8333333333333334, ""YES"": 0.1666666666...","""NO"""
2,100003,es,"@Steven2897 Lee sobre Gamergate, y como eso ha...","{""NO"": 1.0, ""YES"": 0.0}","""NO"""
3,100004,es,@Lunariita7 Un retraso social bastante lamenta...,"{""NO"": 0.5, ""YES"": 0.5}",NaN
4,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,"{""YES"": 0.6666666666666666, ""NO"": 0.3333333333...","""YES"""


In [12]:
t1_train_es_df_sc.rename(columns={'value_ttgt1s_data': 'soft_label', 'value_ttgt1h_data': 'hard_label'}, inplace=True)

## task 2 hard

In [ ]:
t2_train_all_df_sc = train_all_df.loc[:, ['id', 'lang', 'tweet', 'value_ttgt2s_data', 'value_ttgt2h_data']]
t2_train_all_df_sc.head()

,id,lang,tweet,value_ttgt2s_data,value_ttgt2h_data
0,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...","{""REPORTED"": 0.5, ""JUDGEMENTAL"": 0.33333333333...","""REPORTED"""
1,100002,es,@ultimonomada_ Si comicsgate se parece en algo...,"{""NO"": 0.8333333333333334, ""DIRECT"": 0.1666666...","""NO"""
2,100003,es,"@Steven2897 Lee sobre Gamergate, y como eso ha...","{""NO"": 1.0, ""DIRECT"": 0.0, ""REPORTED"": 0.0, ""J...","""NO"""
3,100004,es,@Lunariita7 Un retraso social bastante lamenta...,"{""NO"": 0.5, ""DIRECT"": 0.16666666666666666, ""RE...",NaN
4,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,"{""REPORTED"": 0.16666666666666666, ""NO"": 0.3333...",NaN


In [ ]:
t2_train_all_df_sc.rename(columns={'value_ttgt2s_data': 'soft_label', 'value_ttgt2h_data': 'hard_label'}, inplace=True)

In [ ]:
t2_dev_all_df_sc = dev_all_df.loc[:, ['id', 'lang', 'tweet', 'value_tdgt2s_data', 'value_tdgt2h_data']]
t2_dev_all_df_sc.head()

,id,lang,tweet,value_tdgt2s_data,value_tdgt2h_data
0,300001,es,@Fichinescu La comunidad gamer es un antro de ...,"{""NO"": 0.5, ""JUDGEMENTAL"": 0.3333333333333333,...",NaN
1,300002,es,@anacaotica88 @MordorLivin No me acuerdo de lo...,"{""JUDGEMENTAL"": 0.5, ""REPORTED"": 0.33333333333...","""JUDGEMENTAL"""
2,300003,es,@cosmicJunkBot lo digo cada pocos dias y lo re...,"{""NO"": 1.0, ""DIRECT"": 0.0, ""REPORTED"": 0.0, ""J...","""NO"""
3,300004,es,Also mientras les decia eso la señalaba y deci...,"{""NO"": 0.16666666666666666, ""REPORTED"": 0.6666...","""REPORTED"""
4,300005,es,"And all people killed, attacked, harassed by ...","{""NO"": 0.8333333333333334, ""DIRECT"": 0.1666666...","""NO"""


In [ ]:
t2_dev_all_df_sc.rename(columns={'value_tdgt2s_data': 'soft_label', 'value_tdgt2h_data': 'hard_label'}, inplace=True)

### All

In [ ]:
#relevant functions
import re
# Create function for printing
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)

def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)

def tweet_preprocessor_english(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)

In [ ]:
import optuna
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, XLMRobertaForSequenceClassification, XLMRobertaTokenizer, Trainer
import os.path
from os import path
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

task_encoder = LabelEncoder()

def task_hard_encode(df):
    task_encoder.fit(all_task_2_hard_labels)
    df['hard_label'] = task_encoder.transform(df['hard_label'])
    return df

def task_hard_decode(df):
    df["hard_label"] = task_encoder.inverse_transform(df["hard_label"])
    return df


og_train2 = t2_train_all_df_sc.copy()
og_dev2 = t2_dev_all_df_sc.copy()

all_task_2_hard_labels = pd.concat([og_train2["hard_label"], og_dev2["hard_label"]])

train2_df = task_hard_encode(og_train2)
# print(train1_df.columns)
train2_df = train2_df[["tweet","hard_label"]].dropna()
train2_df = train2_df[train2_df['hard_label'] != 4] #removing nans
train2_df["tweet"] = simple_preprocess(train2_df["tweet"])

test2_df = task_hard_encode(og_dev2)
test2_df = test2_df[["tweet","hard_label"]].dropna()
test2_df = test2_df[test2_df['hard_label'] != 4] #removing nans
test2_df["tweet"] = simple_preprocess(test2_df["tweet"])

print("train2", train2_df.shape)
print(train2_df.head)
print("test2",test2_df.shape)
print(test2_df.head)


import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train2_df, test2_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)




train2 (5496, 2)
<bound method NDFrame.head of                                                   tweet  hard_label
0     @USER Ignora al otro, es un capullo.El problem...           3
1     @USER Si comicsgate se parece en algo a gamerg...           2
2     @USER Lee sobre Gamergate, y como eso ha cambi...           2
5     @USER Aaah sí. Andrew Dobson. El que se dedicó...           2
7     @USER Esta gringa sigue llorando por el gamerg...           0
...                                                 ...         ...
6913  @USER Ma'am if I say that you look like a whor...           0
6915  idk why y’all bitches think having half your a...           0
6916  This has been a part of an experiment with @US...           1
6917  "Take me already" "Not yet. You gotta be ready...           0
6918    @USER why do you look like a whore? /lh HTTPURL           0

[5496 rows x 2 columns]>
test2 (841, 2)
<bound method NDFrame.head of                                                   tweet  hard_labe

In [ ]:
# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4,ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/EXIST 2024/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task2_hard_all'
MAX_LENGTH = 128

def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

In [ ]:
def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task2-hard-all', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

[I 2024-04-14 07:23:22,822] A new study created in memory with name: hp-search-task2-hard-all


Step,Training Loss
500,1.130600
1000,1.070200
1500,1.078600


[I 2024-04-14 07:51:29,422] Trial 0 finished with value: 1.0837610928668835 and parameters: {'learning_rate': 0.00017420128933479402, 'weight_decay': 6.022010235237979e-05, 'num_train_epochs': 3}. Best is trial 0 with value: 1.0837610928668835.


In [ ]:
print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}



Finding study best parameters
----------------------------------------------------------------------------------------------------


Extract best study params
----------------------------------------------------------------------------------------------------
The best learning rate is: 0.00017420128933479402
The best weight decay is: 6.022010235237979e-05
The best epoch is : 3


Create dictionary of the best hyperparameters
----------------------------------------------------------------------------------------------------


### EN

### ES

## task 3 hard

### All

In [ ]:
t3_train_all_df_sc = train_all_df.loc[:, ['id', 'lang', 'tweet', 'value_ttgt3s_data', 'value_ttgt3h_data']]
t3_train_all_df_sc.head()

,id,lang,tweet,value_ttgt3s_data,value_ttgt3h_data
0,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...","{""OBJECTIFICATION"": 0.3333333333333333, ""SEXUA...","[""OBJECTIFICATION"", ""SEXUAL-VIOLENCE""]"
1,100002,es,@ultimonomada_ Si comicsgate se parece en algo...,"{""NO"": 0.8333333333333334, ""OBJECTIFICATION"": ...","[""NO""]"
2,100003,es,"@Steven2897 Lee sobre Gamergate, y como eso ha...","{""NO"": 1.0, ""IDEOLOGICAL-INEQUALITY"": 0.0, ""ST...","[""NO""]"
3,100004,es,@Lunariita7 Un retraso social bastante lamenta...,"{""NO"": 0.5, ""IDEOLOGICAL-INEQUALITY"": 0.5, ""OB...",NaN
4,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,"{""STEREOTYPING-DOMINANCE"": 0.5, ""OBJECTIFICATI...","[""STEREOTYPING-DOMINANCE"", ""OBJECTIFICATION"", ..."


In [ ]:
t3_train_all_df_sc.rename(columns={'value_ttgt3s_data': 'soft_label', 'value_ttgt3h_data': 'hard_label'}, inplace=True)

In [ ]:
t3_dev_all_df_sc = dev_all_df.loc[:, ['id', 'lang', 'tweet', 'value_tdgt3s_data', 'value_tdgt3h_data']]
t3_dev_all_df_sc.head()

,id,lang,tweet,value_tdgt3s_data,value_tdgt3h_data
0,300001,es,@Fichinescu La comunidad gamer es un antro de ...,"{""NO"": 0.5, ""MISOGYNY-NON-SEXUAL-VIOLENCE"": 0....",NaN
1,300002,es,@anacaotica88 @MordorLivin No me acuerdo de lo...,"{""IDEOLOGICAL-INEQUALITY"": 0.3333333333333333,...","[""IDEOLOGICAL-INEQUALITY"", ""STEREOTYPING-DOMIN..."
2,300003,es,@cosmicJunkBot lo digo cada pocos dias y lo re...,"{""NO"": 1.0, ""IDEOLOGICAL-INEQUALITY"": 0.0, ""ST...","[""NO""]"
3,300004,es,Also mientras les decia eso la señalaba y deci...,"{""NO"": 0.16666666666666666, ""SEXUAL-VIOLENCE"":...","[""SEXUAL-VIOLENCE""]"
4,300005,es,"And all people killed, attacked, harassed by ...","{""NO"": 0.8333333333333334, ""STEREOTYPING-DOMIN...","[""NO""]"


In [ ]:
t3_dev_all_df_sc.rename(columns={'value_tdgt3s_data': 'soft_label', 'value_tdgt3h_data': 'hard_label'}, inplace=True)

In [ ]:
t3_train_all_df_sc.head()

,id,lang,tweet,soft_label,hard_label
0,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...","{""OBJECTIFICATION"": 0.3333333333333333, ""SEXUA...","[""OBJECTIFICATION"", ""SEXUAL-VIOLENCE""]"
1,100002,es,@ultimonomada_ Si comicsgate se parece en algo...,"{""NO"": 0.8333333333333334, ""OBJECTIFICATION"": ...","[""NO""]"
2,100003,es,"@Steven2897 Lee sobre Gamergate, y como eso ha...","{""NO"": 1.0, ""IDEOLOGICAL-INEQUALITY"": 0.0, ""ST...","[""NO""]"
3,100004,es,@Lunariita7 Un retraso social bastante lamenta...,"{""NO"": 0.5, ""IDEOLOGICAL-INEQUALITY"": 0.5, ""OB...",NaN
4,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,"{""STEREOTYPING-DOMINANCE"": 0.5, ""OBJECTIFICATI...","[""STEREOTYPING-DOMINANCE"", ""OBJECTIFICATION"", ..."


In [ ]:
t3_train_all_df_sc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6920 entries, 0 to 6919
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          6920 non-null   int64 
 1   lang        6920 non-null   object
 2   tweet       6920 non-null   object
 3   soft_label  6920 non-null   object
 4   hard_label  6050 non-null   object
dtypes: int64(1), object(4)
memory usage: 270.4+ KB


In [ ]:
t3_dev_all_df_sc.head()

,id,lang,tweet,soft_label,hard_label
0,300001,es,@Fichinescu La comunidad gamer es un antro de ...,"{""NO"": 0.5, ""MISOGYNY-NON-SEXUAL-VIOLENCE"": 0....",NaN
1,300002,es,@anacaotica88 @MordorLivin No me acuerdo de lo...,"{""IDEOLOGICAL-INEQUALITY"": 0.3333333333333333,...","[""IDEOLOGICAL-INEQUALITY"", ""STEREOTYPING-DOMIN..."
2,300003,es,@cosmicJunkBot lo digo cada pocos dias y lo re...,"{""NO"": 1.0, ""IDEOLOGICAL-INEQUALITY"": 0.0, ""ST...","[""NO""]"
3,300004,es,Also mientras les decia eso la señalaba y deci...,"{""NO"": 0.16666666666666666, ""SEXUAL-VIOLENCE"":...","[""SEXUAL-VIOLENCE""]"
4,300005,es,"And all people killed, attacked, harassed by ...","{""NO"": 0.8333333333333334, ""STEREOTYPING-DOMIN...","[""NO""]"


In [ ]:
t3_dev_all_df_sc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038 entries, 0 to 1037
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1038 non-null   int64 
 1   lang        1038 non-null   object
 2   tweet       1038 non-null   object
 3   soft_label  1038 non-null   object
 4   hard_label  931 non-null    object
dtypes: int64(1), object(4)
memory usage: 40.7+ KB


In [ ]:
#relevant functions
import re
from sklearn.preprocessing import MultiLabelBinarizer

# Instantiate multi-label encoder
task_encoder = MultiLabelBinarizer()


# Create function for printing
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)


def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets


def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)


def tweet_preprocessor_english(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)


def task_hard_encode(df):
    import numpy as np
    # Create a new DataFrame to store the transformed labels
    transformed_df = df.copy()
    task_encoder.fit(all_task_hard_labels)
    all_labels = []
    for i in range(len(df['hard_label'])):
        # Apply eval to each element of the label column
        all_labels.append(eval(df['hard_label'][i]))

    transformed_labels = task_encoder.transform(all_labels)
    # print(pd.DataFrame(transformed_labels))
    # transformed_df["hard_label"] = pd.DataFrame(transformed_labels)
    transformed_labels = pd.Series([x.tolist() for x in transformed_labels])
    transformed_df["hard_label"] = transformed_labels
    return transformed_df


def task_hard_decode(df):
    # Inverse transform the binary-encoded vectors back to original labels
    import numpy as np
    decoded_df = df.copy()
    decode_labels = np.array(decoded_df["hard_label"].tolist())

    # Inverse transform the label column using the provided task_encoder
    decoded_labels = task_encoder.inverse_transform(decode_labels)

    # Assign the decoded labels back to the DataFrame
    decoded_df["hard_labels"] = decoded_labels

    return decoded_df

In [ ]:
import optuna
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, \
                          XLMRobertaForSequenceClassification, XLMRobertaTokenizer, Trainer
import os.path
from os import path
from datasets import Dataset

og_train3 = t3_train_all_df_sc.copy()
og_dev3 = t3_dev_all_df_sc.copy()

train3_df = og_train3[["tweet","hard_label"]].dropna().reset_index(drop=True)

test3_df = og_dev3[["tweet","hard_label"]].dropna().reset_index(drop=True)

all_task_hard_labels = pd.concat([og_train3["hard_label"],og_dev3["hard_label"]]).dropna()
all_task_hard_labels = all_task_hard_labels.apply(lambda x: eval(x)).tolist()
print(all_task_hard_labels[0])

train3_df = task_hard_encode(train3_df)
# print(train1_df.columns)
train3_df = train3_df[["tweet","hard_label"]].dropna()
train3_df["tweet"] = simple_preprocess(train3_df["tweet"])

test3_df = task_hard_encode(test3_df)
test3_df["tweet"] = simple_preprocess(test3_df["tweet"])

print("train3", train3_df.shape)
print(train3_df.head)
print("test3", test3_df.shape)
print(test3_df.head)

['OBJECTIFICATION', 'SEXUAL-VIOLENCE']
train3 (6050, 2)
<bound method NDFrame.head of                                                   tweet          hard_label
0     @USER Ignora al otro, es un capullo.El problem...  [0, 0, 0, 1, 1, 0]
1     @USER Si comicsgate se parece en algo a gamerg...  [0, 0, 1, 0, 0, 0]
2     @USER Lee sobre Gamergate, y como eso ha cambi...  [0, 0, 1, 0, 0, 0]
3     @USER @USER @USER Entonces como así es el merc...  [1, 0, 0, 1, 0, 1]
4     @USER Aaah sí. Andrew Dobson. El que se dedicó...  [0, 0, 1, 0, 0, 0]
...                                                 ...                 ...
6045  idk why y’all bitches think having half your a...  [0, 1, 0, 1, 1, 1]
6046  This has been a part of an experiment with @US...  [0, 0, 0, 1, 0, 0]
6047  "Take me already" "Not yet. You gotta be ready...  [0, 0, 0, 0, 1, 0]
6048    @USER why do you look like a whore? /lh HTTPURL  [0, 1, 0, 1, 1, 1]
6049  ik when mandy says “you look like a whore” i l...  [0, 0, 0, 1, 0, 1]

[

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train3_df, test3_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6,ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/EXIST 2024/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task3_hard_all'
MAX_LENGTH = 128

In [ ]:
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})


In [ ]:

def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task3-hard-all', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

[I 2024-04-14 09:30:40,679] A new study created in memory with name: hp-search-task3-hard-all


In [ ]:
print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

### EN

### ES

# Construct Optuna - For all tasks soft

## task 1

### All

In [ ]:
#relevant functions
import re
# Create function for printing
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)

def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)

def tweet_preprocessor_english(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)

from sklearn.preprocessing import MultiLabelBinarizer

# Instantiate multi-label encoder

def convert_logits_to_list(logits_dict):
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["YES"], logits_dict["NO"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"YES": logits_list[0], "NO": logits_list[1]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)

import optuna
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset

#load given data
#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task1.csv")
og_dev1 = pd.read_csv("dev_data_task1.csv")

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
# og_train1["soft_label"] = og_train1["soft_label"].apply(check_dtype)
train1_df = og_train1[["tweet","soft_label"]].dropna()


og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
# og_dev1["soft_label"] = og_dev1["soft_label"].apply(check_dtype)
test1_df = og_dev1[["tweet","soft_label"]].dropna()


import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,ignore_mismatched_sizes=True)

# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/ml4nlp2/final_folder/optuna_models/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_hard'
MAX_LENGTH = 128

import torch.nn as nn
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset


# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

def custom_loss_fn(logits, soft_labels):
    probs = F.softmax(logits, dim=1)
    # Apply nn.CrossEntropyLoss
    loss = nn.CrossEntropyLoss(reduction="sum",label_smoothing=0.15)(probs, soft_labels)
    return loss


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

### EN

### ES

## task 2

### All

In [ ]:
#relevant functions
import re
# Create function for printing
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)

def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)

def tweet_preprocessor_english(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)

from sklearn.preprocessing import MultiLabelBinarizer

def convert_logits_to_list(logits_dict):
    """
    order:
    1. classes are
    REPORTED, NO, DIRECT, JUDGEMENTAL
    """
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["REPORTED"], logits_dict["NO"],logits_dict["DIRECT"],logits_dict["JUDGEMENTAL"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"REPORTED": logits_list[0], "NO": logits_list[1],"DIRECT": logits_list[2], "JUDGEMENTAL": logits_list[3]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)

import optuna
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset

#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task2.csv")
og_dev1 = pd.read_csv("dev_data_task2.csv")

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
# og_train1["soft_label"] = og_train1["soft_label"].apply(check_dtype)
train1_df = og_train1[["tweet","soft_label"]].dropna()


og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
# og_dev1["soft_label"] = og_dev1["soft_label"].apply(check_dtype)
test1_df = og_dev1[["tweet","soft_label"]].dropna()

# print("train1",train1_df.shape)
print(train1_df.head)
# print("test1",test1_df.shape)
print(test1_df.head)

import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4,ignore_mismatched_sizes=True)

# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/ml4nlp2/final_folder/optuna_models/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_hard'
MAX_LENGTH = 128

import torch.nn as nn
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset


# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

def custom_loss_fn(logits, soft_labels):
    probs = F.softmax(logits, dim=1)
    # Apply nn.CrossEntropyLoss
    loss = nn.CrossEntropyLoss(reduction="sum",label_smoothing=0.15)(probs, soft_labels)
    return loss


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

### EN

### ES

## task 3

### All

In [ ]:
#relevant functions
import re
# Create function for printing
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)

def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)

def tweet_preprocessor_english(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)

from sklearn.preprocessing import MultiLabelBinarizer

def convert_logits_to_list(logits_dict):
    """
    order:
    1. classes are
    OBJECTIFICATION,'SEXUAL-VIOLENCE',NO,STEREOTYPING-DOMINANCE,IDEOLOGICAL-INEQUALITY,MISOGYNY-NON-SEXUAL-VIOLENCE
    """
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["OBJECTIFICATION"], logits_dict["SEXUAL-VIOLENCE"],logits_dict["NO"],logits_dict["STEREOTYPING-DOMINANCE"],logits_dict["IDEOLOGICAL-INEQUALITY"], logits_dict["MISOGYNY-NON-SEXUAL-VIOLENCE"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"OBJECTIFICATION": logits_list[0], "SEXUAL-VIOLENCE": logits_list[1],"NO": logits_list[2], "STEREOTYPING-DOMINANCE": logits_list[3],"IDEOLOGICAL-INEQUALITY": logits_list[4], "MISOGYNY-NON-SEXUAL-VIOLENCE": logits_list[5]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)


import optuna
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset

#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task3.csv")
og_dev1 = pd.read_csv("dev_data_task3.csv")

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
# og_train1["soft_label"] = og_train1["soft_label"].apply(check_dtype)
train1_df = og_train1[["tweet","soft_label"]].dropna()


og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
# og_dev1["soft_label"] = og_dev1["soft_label"].apply(check_dtype)
test1_df = og_dev1[["tweet","soft_label"]].dropna()

# print("train1",train1_df.shape)
print(train1_df.head)
# print("test1",test1_df.shape)
print(test1_df.head)

import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:

# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6,ignore_mismatched_sizes=True)

In [ ]:
# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/ml4nlp2/final_folder/optuna_models/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_hard'
MAX_LENGTH = 128

In [ ]:
import torch.nn as nn
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset


# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)

In [ ]:
# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

In [ ]:
def custom_loss_fn(logits, soft_labels):
    # Convert probabilities to logits using the inverse sigmoid (logit) function
    # eps = 1e-7  # Small constant to avoid numerical instability
    # logits = torch.log(probabilities.clamp(min=eps, max=1-eps) / (1 - probabilities.clamp(min=eps, max=1-eps)))
    loss = nn.BCEWithLogitsLoss(reduction='sum')(logits, soft_labels)
    return loss

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

In [ ]:
print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

### EN

### ES